In [ ]:
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Candidates / Items : Books

In [ ]:
# books_df = pl.read_csv("data/Books.csv", infer_schema_length=10000)
books_df = pl.read_csv("data/Books.csv", schema_overrides={"Year-Of-Publication": pl.Utf8})
print(f"{len(books_df)=} rows")
print(books_df.head(2))
# print(books_df.describe())
# 280k rows


In [ ]:
# Clean Books data
def pubYear_to_numeric(df):
    
    # Convert publication year to numeric, setting invalid values to null
    df = df.with_columns(
        pl.col("Year-Of-Publication").cast(pl.Float32, strict=False)
    )
    
    return df

In [ ]:
cleaned_books_df = pubYear_to_numeric(books_df)
# print(cleaned_books_df.head(1))
# print(cleaned_books_df.describe())
print("books_df null count")
print(books_df.null_count())
print("cleaned_books_df null count")
print(cleaned_books_df.null_count())


print(f"{len(cleaned_books_df)=} rows")
print(f"after dropping nulls,")
cleaned_books_df = cleaned_books_df.drop_nulls()
print(f"  {len(cleaned_books_df)=} rows")

                          

In [ ]:
# print unique 
print(f"{cleaned_books_df.select('ISBN').n_unique()=}")   
print(f"{cleaned_books_df.select('Publisher').n_unique()=}")     
print(f"{cleaned_books_df.select('Book-Author').n_unique()=}") 

In [ ]:
cleaned_books_df.describe()

In [ ]:

# # Show books with missing publication year
print("Show books with missing Book-Author")
print(cleaned_books_df.filter(pl.col("Book-Author").is_null()))

# # Show books with missing author
print("Show books with missing Publisher")
print(cleaned_books_df.filter(pl.col("Publisher").is_null()))

# show books with missing publisher
print("Show books with missing publication year")
cleaned_books_df.filter(pl.col("Year-Of-Publication").is_null())

In [ ]:
cleaned_books_df = cleaned_books_df.filter(
    # (pl.col("Year-Of-Publication").is_not_null()) &  # Keep non-null values
    (pl.col("Year-Of-Publication") >= 1940) &  # to make normalization easier
    (pl.col("Year-Of-Publication") <= 2025)    # Before or in the near future
)

In [ ]:
cleaned_books_df.describe()

In [ ]:
# # Analyze distribution of book ages
plt.figure(figsize=(10, 6))
cleaned_books_df.select("Year-Of-Publication").to_pandas().hist(bins=50)
plt.title("Distribution of Year-Of-Publication")
plt.xlabel("Year")
plt.ylabel("Count")
plt.show()

In [ ]:
# Clean Books data
def clean_books_data(df):

    
    # Convert publication year to numeric, setting invalid values to null
    df = pubYear_to_numeric(df)
    
    # drop nulls
    df = df.drop_nulls()

    df = df.filter(
        (pl.col("Year-Of-Publication") >= 1940) &  # to make normalization easier
        (pl.col("Year-Of-Publication") <= 2025)    # Before or in the near future
    )

    # Drop unnecessary columns
    df = df.drop(["Image-URL-S", "Image-URL-M"])
    # rename to lowercase with underscore
    df = df.rename({"Book-Title": "book_title", 
                    "Book-Author": "book_author", 
                    "Year-Of-Publication": "year_of_publication", 
                    "Publisher": "publisher",
                    "ISBN": "isbn",
                    "Image-URL-L": "image_url_l",})


    # # Fill missing values
    # df = df.with_columns([
    #     pl.col("Book-Title").fill_null("Unknown Title"),
    #     pl.col("Book-Author").fill_null("Unknown Author"),
    #     pl.col("Publisher").fill_null("Unknown Publisher"),
    #     pl.col("Year-Of-Publication").fill_null(-1)
    #     # For numerical year, use median for missing values
    #     # pl.col("Year-Of-Publication").fill_null(
    #     #     df.select(pl.col("Year-Of-Publication"))
    #     #     .filter(pl.col("Year-Of-Publication").is_not_null())
    #     #     .select(pl.col("Year-Of-Publication").median()).item()
    #     # )
    # ])
    
    # # Extract year features (like book age)
    # current_year = datetime.now().year
    # df = df.with_columns([
    #     (current_year - pl.col("Year-Of-Publication")).alias("Book-age")
    # ])
    
    # # Limit extremely long titles and author names for better processing
    # df = df.with_columns([
    #     pl.col("Book-Title").str.slice(0, 100).alias("Book-Title"),
    #     pl.col("Book-Author").str.slice(0, 100).alias("Book-Author")
    # ])
    
    return df



In [ ]:
clean_books_df = clean_books_data(books_df)
print(clean_books_df.head(2))
print(clean_books_df.describe())

In [ ]:
# check nans
pd_df = clean_books_df.to_pandas()
pd_df["year_of_publication"].isna().sum()

# Customer

In [ ]:
users_df = pl.read_csv("data/users.csv")
print(users_df.head())
print(users_df.describe())
# 280k rows

In [ ]:
# Clean Users data
def clean_users_data(df):
    df = df.rename({"User-ID": "user_id", "Location": "location", "Age": "age"})

    # # Split the location safely
    # df = df.with_columns([
    #     pl.col("Location").fill_null("").str.split(",").alias("Location_Split")
    # ])

    # # Filter out rows where the location does not have exactly 3 parts
    # df = df.filter(pl.col("Location_Split").list.len() != 3)
    # print(df.head(3))

    # # Safely split location (    # Split the location safely
    # df = df.with_columns([
    #     pl.col("Location").str.split(",").alias("Location_Split")
    # ]).with_columns([
    #     pl.col("Location_Split").list.get(0, default="Unknown").alias("City"),
    #     pl.col("Location_Split").list.get(1, default="Unknown").alias("State"),
    #     pl.col("Location_Split").list.get(2, default="Unknown").alias("Country"),
    # ]).drop("Location_Split")  # Drop the temporary split columnrop the temporary split column
    
    # Filter out unreasonable ages (e.g., too young or too old)
    df = df.with_columns([
        pl.when((pl.col("age") < 5) | (pl.col("age") > 100))
        .then(pl.lit(None))
        .otherwise(pl.col("age"))
        .alias("age")
    ])

    median_age = df["age"].median()
    print(f'{median_age=}')


    # Fill missing ages with the median age
    np.random.seed(42) # Set random seed for reproducibility
    df = df.with_columns([
        pl.when(pl.col("age").is_null())
        .then(df["age"].median() + np.random.normal(0, 5, size=df.shape[0]))
        .otherwise(pl.col("age"))
        .alias("age")
    ])

    # # Create age buckets for better feature representation
    # df = df.with_columns([
    #     pl.when(pl.col("age") == -1).then("Unknown")
    #     .when(pl.col("age") < 18).then("Under 18")
    #     .when(pl.col("age") < 25).then("18-24")
    #     .when(pl.col("age") < 35).then("25-34")
    #     .when(pl.col("age") < 45).then("35-44")
    #     .when(pl.col("age") < 55).then("45-54")
    #     .when(pl.col("age") < 65).then("55-64")
    #     .otherwise("65+")
    #     .alias("age-Bucket")
    # ])
   
    # convert user_id from int to string
    df = df.with_columns([
        pl.col("user_id").cast(pl.Utf8),
        pl.col("age").cast(pl.Float32)
    ])
 
    return df

cleaned_users_df = clean_users_data(users_df)
print(cleaned_users_df.head(3))
print(cleaned_users_df.describe())
print(f"n unique User-ID: {cleaned_users_df.select('user_id').n_unique()}")
print(f"n unique Location: {cleaned_users_df.select('location').n_unique()}")


# # Analyze age bucket distribution
# plt.figure(figsize=(12, 6))
# age_counts = cleaned_users_df.group_by("age-Bucket").count().sort("count", descending=True)
# sns.barplot(x=age_counts["age-Bucket"].to_pandas(), y=age_counts["count"].to_pandas())
# plt.title("User Count by age Bucket")
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

In [ ]:
# Analyze age distribution
plt.figure(figsize=(10, 6))
cleaned_users_df.filter(pl.col("age") > -2).select("age").to_pandas().hist(bins=20)
plt.title("Distribution of User ages")
plt.xlabel("age")
plt.ylabel("Count")
plt.show()

# Transactions : Rating

In [ ]:
ratings_df = pl.read_csv("data/ratings.csv")
print(ratings_df.head(3))
print(ratings_df.describe())
# 1.14978e6 rows

# Ratings exploration
print(ratings_df.head(5))
print(ratings_df.columns)
print(ratings_df.dtypes)
print(ratings_df.null_count())

In [ ]:
def clean_ratings_data(df):
    df = df.rename({"User-ID": "user_id", "ISBN": "isbn", "Book-Rating": "rating"})

    # Filter out invalid ratings
    df = df.filter((pl.col("rating") >= 1) & (pl.col("rating") <= 10))

    # convert rating to float
    df = df.with_columns([
        pl.col("rating").cast(pl.Float32),
        pl.col("user_id").cast(pl.Utf8)
    ])

    return df


In [ ]:
cleaned_ratings_df = clean_ratings_data(ratings_df)
print(cleaned_ratings_df.head(3))

In [ ]:
# generate histogram of ratings
plt.figure(figsize=(10, 6))
cleaned_ratings_df.select("rating").to_pandas().hist(bins=10)
plt.title("Distribution of Book Ratings")
plt.xlabel("Rating")
plt.ylabel("Count")
plt.show()

# Retrieval

In [ ]:
# # Create feature sets for two-tower model
# def create_model_features(books_df, users_df, ratings_df, user_stats, book_stats):
#     # Merge books with book stats
#     book_features = books_df.join(book_stats, on="ISBN", how="left")
    
#     # Fill missing stats
#     book_features = book_features.with_columns([
#         pl.col("n_ratings").fill_null(0),
#         pl.col("avg_book_rating").fill_null(0),
#         pl.col("std_book_rating").fill_null(0)
#     ])
    
#     # Merge users with user stats
#     user_features = users_df.join(user_stats, on="User-ID", how="left")
    
#     # Fill missing stats
#     user_features = user_features.with_columns([
#         pl.col("rating_count").fill_null(0),
#         pl.col("avg_rating").fill_null(0),
#         pl.col("std_rating").fill_null(0),
#         pl.col("min_rating").fill_null(0),
#         pl.col("max_rating").fill_null(0)
#     ])
    
#     # Prepare interaction data
#     interactions = ratings_df.select([
#         "User-ID", 
#         "ISBN", 
#         "Book-Rating"
#     ])
    
#     # Create a binary label for implicit feedback
#     interactions = interactions.with_columns([
#         pl.when(pl.col("Book-Rating") > 5)
#         .then(1)
#         .otherwise(0)
#         .alias("positive_interaction")
#     ])
    
#     return book_features, user_features, interactions

# book_features, user_features, interactions = create_model_features(
#     cleaned_books_df, 
#     cleaned_users_df, 
#     processed_ratings_df, 
#     user_stats, 
#     book_stats
# )

# print("Book Features Sample:")
# print(book_features.head())
# print(f"Book Features Shape: {len(book_features)} rows, {len(book_features.columns)} columns")

# print("\nUser Features Sample:")
# print(user_features.head())
# print(f"User Features Shape: {len(user_features)} rows, {len(user_features.columns)} columns")

# print("\nInteractions Sample:")
# print(interactions.head())
# print(f"Interactions Shape: {len(interactions)} rows, {len(interactions.columns)} columns")

# Create Feature Groups for raw, cleaned-up data

In [ ]:
from recsys import hopsworks_integration

In [ ]:
project, fs = hopsworks_integration.feature_store.get_feature_store()

In [ ]:
desc = "Customers data after cleaning and imputation"
users_fg = fs.get_or_create_feature_group(
    name="users",
    description="Book customers data after cleaning and imputation",
    version=1,
    primary_key=["user_id"],
    online_enabled=True,
)
users_fg.insert(cleaned_users_df, wait=True)

In [ ]:
# create books fg
desc = "Books data after cleaning and imputation"
items_fg = fs.get_or_create_feature_group(
    name="items",
    description="Books data after cleaning and imputation",
    version=1,
    primary_key=["isbn"],
    online_enabled=True,
)
items_fg.insert(clean_books_df, wait=True)

In [ ]:
# create ratings fg
desc = "Ratings data after cleaning"
ratings_fg = fs.get_or_create_feature_group(
    name="ratings",
    description="Ratings data after cleaning and imputation",
    version=1,
    primary_key=["user_id", "isbn"],
    online_enabled=True,
)
ratings_fg.insert(cleaned_ratings_df, wait=True)